# 🧠 TinyML - Colab Training Notebook

**Version:** V.1.1.11 (Check commit info below after GitHub update)

This notebook trains your TinyML model on Google Colab GPU.

**Flow:**
1. Check GPU
2. Clone or update your GitHub repo
3. Install Colab-compatible dependencies
4. Run training script (`scripts/train.py`) → saves `src/models/global_model.h5`
5. Run compression pipeline (`compression.py`) → Distillation → Pruning → Quantization → TFLite
6. Save exported models to Google Drive

**Workflow:**
- **Training**: `scripts/train.py` (uses `config/federated_colab.yaml`)
- **Compression**: `compression.py` (full pipeline with all compression techniques)


In [ ]:
# 1. 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# 2. 데이터 폴더 경로 설정
DATA_DIR = "/content/drive/MyDrive/TinyML_models"

## 1️⃣ Runtime & GPU check
Make sure you set **Runtime → Change runtime type → Hardware accelerator → GPU** before running.


In [ ]:
!nvidia-smi || echo "No NVIDIA GPU detected. Please enable GPU in Runtime settings."


## 2️⃣ Clone or update TinyML repository

Set your GitHub repo URL if different.


In [ ]:
import os
import sys
import subprocess

REPO_URL = "https://github.com/danielsoo/TinyML.git"  # change if needed
PROJECT_DIR = "/content/TinyML"

print("🔄 Updating repository from GitHub...")
if not os.path.exists(PROJECT_DIR):
    print(f"📥 Cloning repository from {REPO_URL}...")
    result = subprocess.run(["git", "clone", REPO_URL, PROJECT_DIR], 
                          capture_output=True, text=True, check=True)
    print("✅ Repository cloned successfully")
else:
    print(f"📥 Pulling latest changes from {REPO_URL}...")
    # Change to project directory and pull
    os.chdir(PROJECT_DIR)
    
    # Get current branch name
    branch_result = subprocess.run(["git", "branch", "--show-current"], 
                                   capture_output=True, text=True, check=False)
    current_branch = branch_result.stdout.strip() if branch_result.returncode == 0 else "main"
    if not current_branch:
        current_branch = "main"
    
    print(f"   Current branch: {current_branch}")
    
    # Try test branch first (where files are), then current branch, then main
    # Local uses 'test' branch, so prioritize it
    branches_to_try = ["test"]
    if current_branch not in branches_to_try:
        branches_to_try.append(current_branch)
    if "main" not in branches_to_try:
        branches_to_try.append("main")
    
    # Fetch latest changes from all branches
    fetch_result = subprocess.run(["git", "fetch", "origin", "--all"], 
                                 capture_output=True, text=True, check=False)
    if fetch_result.returncode != 0:
        print(f"⚠️  Git fetch had issues: {fetch_result.stderr[:200]}")
    
    # Try to checkout and pull from the branch
    updated = False
    for branch in branches_to_try:
        print(f"\n   Trying to update from branch: {branch}")
        
        # Checkout the branch
        checkout_result = subprocess.run(["git", "checkout", branch], 
                                        capture_output=True, text=True, check=False)
        
        # Pull latest changes
        pull_result = subprocess.run(["git", "pull", "origin", branch], 
                                    capture_output=True, text=True, check=False)
        
        if pull_result.returncode == 0:
            print(f"   ✅ Successfully updated from {branch}")
            if pull_result.stdout.strip() and "Already up to date" not in pull_result.stdout:
                print(f"   Changes: {pull_result.stdout.strip()[:150]}")
            updated = True
            break
        else:
            print(f"   ⚠️  Could not update from {branch}: {pull_result.stderr[:100]}")
    
    if not updated:
        print(f"⚠️  Could not update from any branch, continuing with current state...")
    
    # Clean up old modelcompression folder if it exists (matching local structure)
    old_compression_dir = Path(PROJECT_DIR) / "src" / "modelcompression"
    if old_compression_dir.exists():
        print(f"\n🧹 Cleaning up old folder structure...")
        print(f"   Removing: {old_compression_dir}")
        import shutil
        shutil.rmtree(old_compression_dir)
        print(f"   ✅ Removed old modelcompression folder")
    
    # Verify critical files exist after update
    compression_dir = Path(PROJECT_DIR) / "src" / "compression"
    required_files = ["distillation.py", "pruning.py", "quantization.py"]
    
    if compression_dir.exists():
        files_in_dir = [f.name for f in compression_dir.iterdir() if f.is_file()]
        missing = [f for f in required_files if f not in files_in_dir]
        if missing:
            print(f"\n⚠️  WARNING: After GitHub update, files still missing: {missing}")
            print(f"   Files in {compression_dir}: {files_in_dir}")
            print(f"   Current branch: {current_branch}")
            print(f"   Please ensure files are committed and pushed to the 'test' branch.")
    else:
        print(f"\n⚠️  WARNING: {compression_dir} does not exist after GitHub update!")
        print(f"   This indicates the folder structure may not be in the current branch.")

os.chdir(PROJECT_DIR)

# Show current commit information
print("\n📌 Repository Version Info:")
commit_hash_result = subprocess.run(["git", "rev-parse", "--short", "HEAD"], 
                                    capture_output=True, text=True, check=False)
commit_msg_result = subprocess.run(["git", "log", "-1", "--pretty=format:%s"], 
                                   capture_output=True, text=True, check=False)
commit_date_result = subprocess.run(["git", "log", "-1", "--pretty=format:%ad", "--date=short"], 
                                    capture_output=True, text=True, check=False)

if commit_hash_result.returncode == 0:
    commit_hash = commit_hash_result.stdout.strip()
    commit_msg = commit_msg_result.stdout.strip() if commit_msg_result.returncode == 0 else "N/A"
    commit_date = commit_date_result.stdout.strip() if commit_date_result.returncode == 0 else "N/A"
    print(f"   Commit Hash: {commit_hash}")
    print(f"   Commit Message: {commit_msg}")
    print(f"   Commit Date: {commit_date}")
else:
    print("   ⚠️  Could not retrieve commit information")

# Add project directory to Python path for module imports
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

print(f"\n✅ Project directory ready: {PROJECT_DIR}")


## 2️⃣.5 Update config data path
Google Drive에 있는 데이터 경로(`DATA_DIR`)를 `config/federated_colab.yaml`에 반영합니다.



In [ ]:
import yaml
from pathlib import Path

config_path = Path(PROJECT_DIR) / "config" / "federated_colab.yaml"

if config_path.exists():
    with config_path.open("r") as f:
        cfg = yaml.safe_load(f)

    cfg.setdefault("data", {})
    cfg["data"]["path"] = DATA_DIR

    with config_path.open("w") as f:
        yaml.safe_dump(cfg, f, sort_keys=False, allow_unicode=True)

    print(f"Updated federated_colab.yaml data.path -> {cfg['data']['path']}")
else:
    raise FileNotFoundError(f"Cannot find {config_path}")



## 3️⃣ Generate Colab-specific requirements (no macOS-only packages)

We remove `tensorflow-macos` and `tensorflow-metal` from `requirements.txt` automatically.


In [ ]:
import os

src_req = "requirements.txt"
colab_req = "colab_requirements.txt"

# Skip packages that are preinstalled or incompatible on Colab (tensorflow/numpy variants)
skip_keywords = ["tensorflow", "numpy", "tensorflow-macos", "tensorflow-metal"]

if os.path.exists(src_req):
    with open(src_req, "r") as f:
        lines = f.readlines()

    with open(colab_req, "w") as f:
        for line in lines:
            if any(kw in line for kw in skip_keywords):
                continue
            f.write(line)

    print("Generated:", colab_req)
    with open(colab_req, "r") as f:
        print(f.read())
else:
    print("No requirements.txt found. Skipping Colab requirements generation.")


## 4️⃣ Install dependencies (Colab compatible)

- Installs from `colab_requirements.txt` if present.
- Installs standard `tensorflow` for Linux GPU.


In [ ]:
import os
import subprocess

print("📦 Installing dependencies...")

if os.path.exists("colab_requirements.txt"):
    print("   Installing from colab_requirements.txt...")
    subprocess.run(["pip", "install", "-r", "colab_requirements.txt"], check=True)
else:
    print("⚠️  colab_requirements.txt not found. Install your packages manually if needed.")

print("   Installing Flower...")
subprocess.run(["pip", "install", "flwr[simulation]"], check=True)

# Fix protobuf compatibility issue with TensorFlow 2.19.0
# TensorFlow 2.19.0 requires protobuf==3.20.3, but Colab may have newer version
print("\n🔧 Fixing protobuf compatibility (TensorFlow 2.19.0 requires protobuf==3.20.3)...")
subprocess.run(["pip", "install", "--force-reinstall", "protobuf==3.20.3"], check=True)
print("✅ Protobuf fixed")

In [ ]:
# Fix protobuf compatibility issue (if needed)
# If you see AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
# Uncomment the following line:
# !pip install protobuf==3.20.3

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))

# Check protobuf version
try:
    import google.protobuf
    print("Protobuf version:", google.protobuf.__version__)
except:
    pass


## 5️⃣ (Optional) Download or prepare dataset

Edit this cell if your training script expects data in a specific path.
For example, you can mount Google Drive or download from Kaggle here.


In [ ]:
# Example: mount Google Drive if your data is stored there.
# from google.colab import drive
# drive.mount('/content/drive')

# Example: create a data directory
# os.makedirs('data', exist_ok=True)
# Then copy or download your dataset into ./data

print("Dataset preparation step: customize as needed.")


## 6️⃣ Run training

This cell tries to run `train.py` at repo root.
If your main script is at a different path, edit accordingly (e.g. `src/train.py`).


In [ ]:
import pandas as pd
from pathlib import Path

data_dir = Path("/content/drive/MyDrive/TinyML_models")  # CSV들이 있는 경로로 수정
csv_paths = sorted(data_dir.glob("*.csv"))

dfs = [pd.read_csv(p, low_memory=False) for p in csv_paths]
df = pd.concat(dfs, ignore_index=True)

print("Total Samples:", len(df))
print(df["attack"].value_counts())
print(df["attack"].value_counts(normalize=True))  # 비율 확인

In [ ]:
import os
import sys
from datetime import datetime

PROJECT_DIR = "/content/TinyML"
os.chdir(PROJECT_DIR)

# Add project directory to Python path
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

# Use train.py script (unified training script)
# This will automatically detect Colab environment and use federated_colab.yaml
print("🚀 Running training with scripts/train.py...")
print("   This will use config/federated_colab.yaml automatically\n")

import subprocess
result = subprocess.run(
    [sys.executable, "scripts/train.py", "--config", "config/federated_colab.yaml"],
    cwd=PROJECT_DIR,
    check=False
)

# Check if model was actually saved
model_path = Path(PROJECT_DIR) / "src" / "models" / "global_model.h5"
model_exists = model_path.exists() and model_path.stat().st_size > 0

if result.returncode == 0 or model_exists:
    print("\n✅ Training complete!")
    print("   Model saved to: src/models/global_model.h5")
    if result.returncode != 0:
        print("   ⚠️  Note: Training process returned non-zero exit code, but model was saved.")
else:
    print("\n❌ Training failed!")
    print(f"   Exit code: {result.returncode}")
    print("   Model file not found.")


## 7️⃣.5 Compression Pipeline (Full Pipeline)

Run the complete compression pipeline: Distillation → Pruning → Quantization → TFLite Export.

This uses the trained model from the previous step and applies all compression techniques.


In [ ]:
# Run full compression pipeline
import os
import sys
from pathlib import Path

PROJECT_DIR = "/content/TinyML"
os.chdir(PROJECT_DIR)

# Add project directory to Python path
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

# Ensure __init__.py exists in src/compression/ directory
compression_init = Path(PROJECT_DIR) / "src" / "compression" / "__init__.py"
if not compression_init.exists():
    print("📝 Creating src/compression/__init__.py...")
    compression_init.parent.mkdir(parents=True, exist_ok=True)
    compression_init.write_text('"""\nCompression module for TinyML model compression techniques.\n\nThis module provides:\n- Knowledge Distillation\n- Structured Pruning\n- Quantization\n"""\n')
    print("✅ Created __init__.py\n")

# Check if model exists
model_path = "src/models/global_model.h5"
if not os.path.exists(model_path):
    print("⚠️  WARNING: No trained model found!")
    print("⚠️  Please run the training step (Cell 17) first.")
    print("⚠️  The compression script will train a quick test model, but results will be less accurate.")
    print()

# Run compression pipeline
print("🚀 Running compression pipeline...")
print("   This will run:")
print("   1. Test 1: Full pipeline (Train → Distillation → Pruning → Quantization → TFLite)")
print("   2. Test 2: Saved model compression (Load → Prune → Quantize → TFLite)")
print()

# Run compression.py - Ensure proper path setup (matching local structure)
os.chdir(PROJECT_DIR)

# CRITICAL: Ensure project root is at the BEGINNING of sys.path
if PROJECT_DIR in sys.path:
    sys.path.remove(PROJECT_DIR)
sys.path.insert(0, PROJECT_DIR)

# Expected structure (matching local):
# /content/TinyML/
#   compression.py (root)
#   src/
#     compression/
#       __init__.py
#       distillation.py
#       pruning.py
#       quantization.py

compression_dir = Path(PROJECT_DIR) / "src" / "compression"
required_files = ["distillation.py", "pruning.py", "quantization.py"]

print(f"📁 Current directory: {os.getcwd()}")
print(f"📁 Project directory: {PROJECT_DIR}")
print(f"📁 Expected compression dir: {compression_dir}")
print(f"📁 src/compression exists: {compression_dir.exists()}")

# Verify structure (matching local exactly)
if not compression_dir.exists():
    print(f"❌ ERROR: {compression_dir} not found!")
    print(f"   Please ensure GitHub repository is up to date.")
    raise FileNotFoundError(f"Compression directory not found: {compression_dir}")

# Check required files
files_in_dir = [f.name for f in compression_dir.iterdir() if f.is_file()]
print(f"📁 Files in src/compression: {files_in_dir}")

missing_files = [f for f in required_files if f not in files_in_dir]
if missing_files:
    print(f"\n❌ ERROR: Missing required files: {missing_files}")
    print(f"   Please ensure GitHub repository is up to date.")
    print(f"   Expected files in {compression_dir}:")
    for req_file in required_files:
        status = "✅" if req_file in files_in_dir else "❌"
        print(f"   {status} {req_file}")
    raise FileNotFoundError(f"Missing files: {missing_files}")

# Ensure __init__.py exists
init_file = compression_dir / "__init__.py"
if not init_file.exists():
    print(f"📝 Creating {init_file.name}...")
    init_file.write_text('"""\nCompression module for TinyML model compression techniques.\n\nThis module provides:\n- Knowledge Distillation\n- Structured Pruning\n- Quantization\n"""\n')

# Verify compression.py exists at root
compression_script = Path(PROJECT_DIR) / "compression.py"
if not compression_script.exists():
    print(f"❌ ERROR: compression.py not found at {compression_script}")
    print(f"   This file should be at the project root.")
    raise FileNotFoundError(f"compression.py not found: {compression_script}")

# Test imports before running
print(f"\n🔍 Testing imports...")
try:
    import src.compression.distillation
    import src.compression.pruning
    import src.compression.quantization
    print("✅ Successfully imported all compression modules")
except ImportError as e:
    print(f"❌ Failed to import compression modules: {e}")
    print(f"   sys.path (first 5): {sys.path[:5]}")
    print(f"   compression_dir: {compression_dir}")
    raise

# Run compression.py
print(f"\n🚀 Running compression.py...")
try:
    import runpy
    original_argv = sys.argv.copy()
    sys.argv = ["compression.py"]
    
    # Run compression.py
    runpy.run_path(str(compression_script), run_name="__main__")
    
    sys.argv = original_argv
    print("\n✅ Compression pipeline complete!")
    print("   Check outputs/test_pipeline/ and models/tflite/ for compressed models")
            
except Exception as e:
    import traceback
    if 'original_argv' in locals():
        sys.argv = original_argv
    print("\n❌ Compression pipeline failed with error:")
    print(f"   {type(e).__name__}: {e}")
    print("\nFull traceback:")
    traceback.print_exc()
    raise


## 7️⃣.6 FGSM Adversarial Attack Testing

Test FGSM (Fast Gradient Sign Method) attack on the trained and compressed models.
This evaluates the model's robustness against adversarial examples.

**Note:** This can be run on both the original model and compressed models.

In [ ]:
# Run FGSM attack testing
import os
import sys

PROJECT_DIR = "/content/TinyML"
os.chdir(PROJECT_DIR)

# Add project directory to Python path
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

# Check if model exists
model_path = "src/models/global_model.h5"
if not os.path.exists(model_path):
    print("⚠️  WARNING: No trained model found!")
    print("⚠️  Please run the training step (Cell 17) first.")
    print("⚠️  The script will train a quick test model, but results will be less accurate.")
    print()

# Run FGSM attack test
print("Running FGSM attack testing...")
print("This may take a few minutes depending on dataset size.\n")

!python scripts/test_fgsm_attack.py

## 7️⃣ Save trained model(s) to Google Drive

This will look for common output filenames (e.g. `tiny_model.tflite`) in the project root and copy them to your Drive.
Edit `OUTPUT_FILES` if your script uses different names or locations.


## 8️⃣ Compression Analysis

Analyze model size, accuracy, and inference speed at each compression stage.
Generate visualizations and reports.



In [ ]:
# Export trained model to TFLite (optional, for comparison)
import tensorflow as tf
import yaml

# Load config
with open("config/federated_colab.yaml") as f:
    cfg = yaml.safe_load(f)

# Load trained model
model_path = "src/models/global_model.h5"
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)
    
    # Export to TFLite
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    
    # Save
    tflite_path = "src/models/global_model.tflite"
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)
    print(f"✅ Saved TFLite model: {tflite_path}")
else:
    print(f"⚠️ Model not found: {model_path}")



In [ ]:
# Run compression analysis
import os
import sys

PROJECT_DIR = "/content/TinyML"
os.chdir(PROJECT_DIR)

# Add project directory to Python path
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

# Analyze models - 두 모델의 압축 결과 모두 포함
models_to_analyze = []

# 1. Train.py로 학습한 모델 (Federated Learning)
# Original model
if os.path.exists("src/models/global_model.h5"):
    models_to_analyze.append("Federated_Original:src/models/global_model.h5")

if os.path.exists("src/models/global_model.tflite"):
    models_to_analyze.append("Federated_Original_TFLite:src/models/global_model.tflite")

# Compression.py에서 압축한 train.py 모델
if os.path.exists("models/tflite/saved_model_original.tflite"):
    models_to_analyze.append("Federated_Compressed_Original:models/tflite/saved_model_original.tflite")

if os.path.exists("models/tflite/saved_model_pruned_quantized.tflite"):
    models_to_analyze.append("Federated_Compressed_PrunedQuantized:models/tflite/saved_model_pruned_quantized.tflite")

# 2. Compression.py 자체 학습 모델 (Self-Trained)
# Compression pipeline에서 생성된 모델들
if os.path.exists("outputs/test_pipeline/original_float32.tflite"):
    models_to_analyze.append("SelfTrained_Original:outputs/test_pipeline/original_float32.tflite")

if os.path.exists("outputs/test_pipeline/pruned_float32.tflite"):
    models_to_analyze.append("SelfTrained_Pruned:outputs/test_pipeline/pruned_float32.tflite")

if os.path.exists("outputs/test_pipeline/pruned_quantized_int8.tflite"):
    models_to_analyze.append("SelfTrained_PrunedQuantized:outputs/test_pipeline/pruned_quantized_int8.tflite")

if not models_to_analyze:
    print("⚠️  No models found for analysis. Make sure:")
    print("   1. Training step (Cell 17) completed")
    print("   2. Compression pipeline (Cell 21) completed")
    print()
else:
    print(f"📊 Found {len(models_to_analyze)} model(s) to analyze:")
    for model in models_to_analyze:
        print(f"   - {model}")
    print()

    models_str = " ".join([f'"{m}"' for m in models_to_analyze])

    # Use first original model as baseline
    baseline = None
    if os.path.exists("src/models/global_model.h5"):
        baseline = "src/models/global_model.h5"
    elif os.path.exists("outputs/test_pipeline/original_float32.tflite"):
        baseline = "outputs/test_pipeline/original_float32.tflite"

    cmd = f"""python scripts/analyze_compression.py \
        --models {models_str} \
        --config config/federated_colab.yaml \
        --output-dir data/processed/analysis \
        --format all"""

    if baseline:
        cmd += f" \\\n        --baseline {baseline}"

    print("Running compression analysis...")
    print(f"Command: {cmd}\n")
    print(f"Python path: {sys.path[:3]}...\n")
    !{cmd}



In [ ]:
# Generate visualizations
import os
import sys

PROJECT_DIR = "/content/TinyML"
os.chdir(PROJECT_DIR)

# Add project directory to Python path (if not already added)
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

results_path = "data/processed/analysis/compression_analysis.csv"

if os.path.exists(results_path):
    print("Generating visualizations...")
    !python scripts/visualize_results.py \
        --results {results_path} \
        --output-dir data/processed/analysis \
        --plot all
else:
    print(f"⚠️ Results file not found: {results_path}")



In [ ]:
# Display visualizations inline
from IPython.display import Image, display
import os

analysis_dir = "data/processed/analysis"
plots = [
    "size_vs_accuracy.png",
    "compression_metrics.png",
    "compression_ratio.png"
]

for plot in plots:
    plot_path = os.path.join(analysis_dir, plot)
    if os.path.exists(plot_path):
        print(f"\n## {plot}")
        display(Image(plot_path))
    else:
        print(f"⚠️ Plot not found: {plot_path}")



In [ ]:
# Copy analysis results to Google Drive
import shutil
from google.colab import drive

drive.mount('/content/drive')

analysis_dir = "data/processed/analysis"
drive_dir = "/content/drive/MyDrive/TinyML_models/analysis"

# Create directory
os.makedirs(drive_dir, exist_ok=True)

# Copy all analysis files
found_any = False
if os.path.exists(analysis_dir):
    for file in os.listdir(analysis_dir):
        src = os.path.join(analysis_dir, file)
        dst = os.path.join(drive_dir, file)
        if os.path.isfile(src):
            shutil.copy(src, dst)
            print(f"✅ Copied: {file}")
            found_any = True

if found_any:
    print(f"\n✅ All analysis results saved to: {drive_dir}")
else:
    print(f"⚠️ No analysis files found in {analysis_dir}")



## 8️⃣.5 Save Compression Pipeline Outputs to Google Drive

Copy compressed models from compression.py to Google Drive.

In [ ]:
# Copy compression pipeline outputs to Google Drive
import os
import shutil
from google.colab import drive
from pathlib import Path

PROJECT_DIR = "/content/TinyML"
drive.mount('/content/drive')

dest_dir = "/content/drive/MyDrive/TinyML_models"
os.makedirs(dest_dir, exist_ok=True)

# Compression pipeline outputs
compression_dirs = [
    "outputs/test_pipeline",
    "models/tflite"
]

found_any = False
for comp_dir in compression_dirs:
    src_dir = os.path.join(PROJECT_DIR, comp_dir)
    if os.path.exists(src_dir):
        dst_dir = os.path.join(dest_dir, comp_dir)
        os.makedirs(dst_dir, exist_ok=True)
        
        # Copy all files in the directory
        for file in os.listdir(src_dir):
            src_path = os.path.join(src_dir, file)
            dst_path = os.path.join(dst_dir, file)
            if os.path.isfile(src_path):
                shutil.copy(src_path, dst_path)
                print(f"✅ Copied {comp_dir}/{file}")
                found_any = True

if found_any:
    print(f"\n✅ All compression outputs saved to: {dest_dir}/")
    print("   📌 Compressed models: outputs/test_pipeline/ and models/tflite/")
else:
    print("⚠️ No compression outputs found. Make sure compression.py (Cell 21) ran successfully.")

In [ ]:
import os
import shutil
from google.colab import drive
from pathlib import Path

PROJECT_DIR = "/content/TinyML"
OUTPUT_FILES = [
    "src/models/global_model.h5",  # Latest model (always copied)
    "src/models/global_model.tflite",
    "models/global_model.h5",
    "models/global_model.tflite",
    "tiny_model.tflite",
    "model.tflite",
    "model.h5",
    "saved_model.pb"
]

# Also find all timestamped models to preserve history
models_dir = Path(PROJECT_DIR) / "src" / "models"
if models_dir.exists():
    timestamped_models = list(models_dir.glob("global_model_*.h5"))
    if timestamped_models:
        OUTPUT_FILES.extend([f"src/models/{f.name}" for f in timestamped_models])
        print(f"📦 Found {len(timestamped_models)} timestamped model(s) to preserve")

# Mount Drive to store the trained models
drive.mount('/content/drive')

dest_dir = "/content/drive/MyDrive/TinyML_models"
os.makedirs(dest_dir, exist_ok=True)

# Create src/models subdirectory in Drive to preserve structure
drive_src_dir = os.path.join(dest_dir, "src", "models")
os.makedirs(drive_src_dir, exist_ok=True)

found_any = False
for fname in OUTPUT_FILES:
    src_path = os.path.join(PROJECT_DIR, fname)
    if os.path.exists(src_path):
        # Keep directory structure in Drive
        dst_path = os.path.join(dest_dir, fname)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        shutil.copy(src_path, dst_path)
        print(f"✅ Copied {fname}")
        found_any = True

if not found_any:
    print("⚠️ No known model files found. Make sure your training script saves a model and update OUTPUT_FILES if needed.")
else:
    print(f"\n✅ All models saved to: {dest_dir}/src/models/")
    print("   📌 Timestamped models are preserved (no overwriting)")
    print("   📌 Latest model: global_model.h5")
